In [1]:
#import libraries in python3 kernel
import pandas as pd
import seaborn as sns
import glob
import os
import boto
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import numpy as np
import skbio
#import fastcluster #this thing is tricky for clustermap when not all relationship exists
from functools import reduce
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.SeqUtils import GC
from collections import defaultdict
from collections import Counter
import statistics
import itertools as it
from scipy import stats
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline

In [2]:
#paths to files needed as inputs for respective functions
all_merged_path_18s = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/merged_all_18s_tables.tsv'
all_merged_path_16s = '/Users/Diana/Documents/escuela/phd/plugin_paper/final_data_and_code/input_data/merged_all_tables_16s.tsv'
manifestfile_path = '/Users/Diana/Documents/escuela/phd/plugin_paper/final_data_and_code/input_data/MANIFEST.tsv'
metadatafile_path = '/Users/Diana/Documents/escuela/phd/plugin_paper/final_data_and_code/input_data/METADATA.tsv'
path_to_tsvs_18s = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/all_trims/all_taxonomies/'
path_to_tsvs_16s = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/all_taxonomies/'
path_to_bacaro = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/TD_metric/Bacaros_Beta/all18sstagg_s_lvl7/all_18s_stagg_taxo.csv'
expected_file = 'expected_18s_staggered'
path_to_expected_18s_stag = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/IN-SILICO/in-silico-mocks/18s_stagg_even_expected_tax.tsv'
path_to_expected_18s_even = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/IN-SILICO/in-silico-mocks/18s_stagg_even_expected_tax.tsv'
path_to_expected_16s_stag = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/IN-SILICO/in-silico-mocks/18s_stagg_even_expected_tax.tsv'
path_to_expected_16s_even = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/IN-SILICO/in-silico-mocks/18s_stagg_even_expected_tax.tsv'
path_to_expected_16s = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/IN-SILICO/in-silico-mocks/18s_stagg_even_expected_tax.tsv'



path_to_fastQC_forward = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/02-EUKs/run46/Forward/merged_f_fastqc/avg_base_q_forward.txt'
path_to_fastQC_reverse = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/02-EUKs/run46/Reverse/merged_r_fastqc/avg_base_q_reverse.txt'
path_to_best_taxo = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/all_trims/separated_taxonomies/Staggered/46F260R120.tsv'
path_to_compare = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/all_trims/separated_taxonomies/Staggered/46F20R0.tsv'
path_to_merged_18s_fasta = '/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/all_trims/merged.fasta'
blast_results = '/Users/Diana/Documents/escuela/phd/plugin_paper/final_data_and_code/analyse_data/BLAST/blast_results.txt'

In [3]:
paths_to_lvls_18s = {
#    "Domain": ""
  "Phylum": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/TD_metric/Bacaros_Beta/all18slvl2/all_18s_stagg_taxo.csv",
  "Class": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/TD_metric/Bacaros_Beta/all18slvl3/all_18s_stagg_taxo.csv",
  "Order": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/TD_metric/Bacaros_Beta/all18slvl4/all_18s_stagg_taxo.csv",
  "Family": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/TD_metric/Bacaros_Beta/all18slvl5/all_18s_stagg_taxo.csv",
  "Genus": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/TD_metric/Bacaros_Beta/all_18s_stagg_lvl6/all_18s_stagg_taxo.csv",
  "Species": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/TD_metric/Bacaros_Beta/all_18s_stagg_lvl7/all_18s_stagg_taxo.csv"
}

In [4]:
paths_to_lvls_16s = {
    "Domain": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/Bacaros_Beta/results-1/all_taxonomies.csv",
  "Phylum": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/Bacaros_Beta/results-2/all_taxonomies.csv",
  "Class": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/Bacaros_Beta/results-3/all_taxonomies.csv",
  "Order": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/Bacaros_Beta/results-4/all_taxonomies.csv",
  "Family": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/Bacaros_Beta/results-5/all_taxonomies.csv",
  "Genus": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/Bacaros_Beta/results-6/all_taxonomies.csv",
  "Species": "/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/16S/02-PROKs/Bacaros_Beta/results-7/all_taxonomies.csv"
}

In [5]:
id_keys = pd.DataFrame(np.array([['in.silico.stag_1', 'Acineta_flava_KR-10010701_Acinetidae_X_sp._strain5', 'Ciliophora; Phyllopharyngea; Suctoria'],
                       ['in.silico.stag_81', 'Amoebophrya_sp._Amoebophrya_sp.', 'Dinophyta; Syndiniales; Dino-Group-II_a'],
                       ['in.silico.stag_721', 'Amoebophrya_sp._Dino-Group-II-Clade-10-and-11_X_sp.', 'Dinophyta; Syndiniales; Dino-Group-II_b'],
                       ['in.silico.stag_1041', 'Ceratium_longipes_ccmp1770_Ceratium_tenue', 'Dinophyta; Dinophyceae; Neoceratium'],
                       ['in.silico.stag_1043', 'Chrysochromulina_simplex_partial_Chrysochromulina_X_sp.', 'Haptophyta; Prymnesiophyceae; Prymnesiales'],
                       ['in.silico.stag_1723', 'Cryothecomonas_sp._APCC_Cryothecomonas_sp.', 'Cercozoa; Filosa-Thecofilosea; Cryomonadida'],
                       ['in.silico.stag_2043', 'Guinardia_delicatula_Guinardia_delicatula', 'Ochrophyta; Bacillariophyta; Coscinodiscophyceae_a'],
                       ['in.silico.stag_2523', 'Gymnodinium_sp._Dinophyceae', 'Dinophyta; Dinophyceae; Gyrodinium'],
                       ['in.silico.stag_2803', 'Gymnodinium_sp._Gymnodinium_dorsalisulcum','Dinophyta; Dinophyceae; Gymnodinium'],
                       ['in.silico.stag_3083', 'Larcopyle_butschlii_Larcopyle_butschlii','Radiolaria; RAD-B; Larcopyle'],
                       ['in.silico.stag_3283', 'Leptocylindrus_convexus_SZN-B768_Radial-centric-basal-Coscinodiscophyceae_X_sp.','Ochrophyta; Bacillariophyta; Coscinodiscophyceae_b'],
                       ['in.silico.stag_3337', 'Lingulodinium_polyedrum_Lingulodinium_polyedrum','Dinophyta; Dinophyceae; Lingulodinium'],
                       ['in.silico.stag_3737', 'Paracalanus_parvus_Paracalanus_parvus','Metazoa; Arthropoda; Crustacea'],
                       ['in.silico.stag_3741', 'Strombidium_cf._basimorphum_Strombidium_basimorphum','Ciliophora; Spirotrichea; Oligotrichia'],
                       ['in.silico.stag_3781', 'Woloszynskia_cincta_Woloszynskia_cincta','Dinophyta; Dinophyceae; Dinophyceae_X'],
                       ['in.silico.stag_3801', 'Karlodinium_micrum_clone_Dino-Group-I-Clade-5_X_sp.','Dinophyta; Syndiniales; Dino-Group-I'],
                      ]),
             columns=['sseqid', 'taxonomic_id', 'short_id'])

In [32]:
def divide_by_comm(community, composition, runnumber, R='all', F='all'):
    df = pd.read_csv('input_data/'+community+'/merged_all_tables.tsv', sep='\t')
    tables = df[['sample_name', 'feature_id', 'feature_frequency']].copy()
    tables.rename(columns={'sample_name':'file'}, inplace=True)
    manifest = pd.read_csv('input_data/MANIFEST.tsv', sep='\t')
    manifest['file'] = [s.split('SPOT_USC_2/')[1] for s in manifest['absolute-filepath']]
    manifest['file'] = [s.split('.R')[0] for s in manifest['file']]
    manifest = manifest.drop(columns = ['absolute-filepath', 'direction'])
    manifest.drop_duplicates()
    merged = pd.merge(tables,manifest, on='file')
    merged = merged.drop(columns = ['file'])
    merged = merged.drop_duplicates()
    print('Set up manifest ...')
    metadata = pd.read_csv('input_data/METADATA.tsv', sep='\t')
    merged = pd.merge(merged,metadata, on='sample-id')
    merged = merged.replace({'V2': '16S'}, regex=True)

    print('Set up metadata ...')
    merged.to_csv('filtering_asvs.tsv', sep = '\t')
    print('Saved filtering_asvs.tsv')
    
    
#make df of features/composition+run+comm
    files = glob.glob('{0}*.tsv'.format('input_data/'+community+'/all_taxonomies_'+community+'/'))
    taxos = []
#    if not os.path.exists(path+composition):
#        os.mkdir(path+composition)
    for filename in files:
        tax = pd.read_csv(filename, sep='\t')
        tax['table_id'] = str(filename.split('/')[-1])
        tax["table_id"] = tax["table_id"].str.replace(".tsv", "")
        tax['Forward_trim'], tax['Reverse_trim'] = tax['table_id'].str.split('R', 1).str
        tax['Forward_trim'] = tax['Forward_trim'].map(lambda x: x.lstrip('F'))
        tax["Forward_trim"] = pd.to_numeric(tax["Forward_trim"])
        tax["Reverse_trim"] = pd.to_numeric(tax["Reverse_trim"])
        taxos.append(tax)
    print('Appended all taxonomies to taxos')
    taxos = pd.concat(taxos)
    taxos = taxos.rename(columns={"Feature ID": "feature_id"}, errors="raise")
    separated = taxos.merge(merged, how='left', on='feature_id')
    separated = separated.drop_duplicates()
    separated = separated[separated["community"] == community]
    separated = separated[separated["composition"] == composition]
    separated['run-number']= separated['run-number'].astype(str)
    separated = separated[separated["run-number"] == runnumber]
    separated['sum'] = separated.groupby(['table_id','sample-id'])['feature_frequency'].transform('sum')
    separated['ratio'] = separated['feature_frequency']/(separated['sum'])
    #make a dictionary with keys for id-ing the taxon belonging to this sub-community
    separated_dic = pd.Series(separated.Taxon.values,separated.feature_id.values).to_dict()

#generate folder of split taxonomies by runnumber and composition
    # Directory
    directory = composition+runnumber
    # Parent Directory path
    parent_dir = 'input_data/'+community+'/all_taxonomies_'+community
    # Path
    path = os.path.join(parent_dir, directory)
    # Create the directory
    # 'GeeksForGeeks' in
    # '/home / User / Documents'
    os.mkdir(path)
    for filename in files:
        taxonomy = pd.read_csv(filename, sep='\t')
        taxonomy = taxonomy.rename(columns={"Feature ID": "feature_id"}, errors="raise")
        newz = taxonomy.merge(merged, how='left', on='feature_id')
        #new = newz.drop(['sample-id'], axis=1)
        new = newz.drop_duplicates()
        new = new[new["community"] == community]
        new = new[new["composition"] == composition]
        new['run-number']= new['run-number'].astype(str)
        new = new[new["run-number"] == runnumber]
        new = new[new.feature_frequency != 0]
        new = new.rename(columns={"feature_id":"Feature ID"}, errors="raise")
        new = new[['Feature ID', 'Taxon', 'Confidence']].copy()
        new = new.drop_duplicates()
        new.to_csv(filename.split('all_taxonomies_'+community)[0]+'all_taxonomies_'+community+'/'+composition+runnumber+'/'+runnumber+filename.split('all_taxonomies_'+community+'/')[1], sep = '\t') 
    
    # Get list of all .qza
    repseqs = glob.glob('{0}*/*/representative_sequences.qza'.format('/Users/Diana/Documents/escuela/phd/plugin_paper/mock_code/18S/all_trims/'), recursive=True)
    for repseq in repseqs:
        with ZipFile(repseq, 'r') as zipObj:
            # Get a list of all archived file names from the zip
            listOfFileNames = zipObj.namelist()
            # Iterate over the file names
            for fileName in listOfFileNames:
                # Check filename endswith fasta
                if fileName.endswith('.fasta'):
                    # Extract a single file from zip
                    zipObj.extract(fileName, 'temp_fasta')
                    
    fastaoutfilename = 'merged'+community+composition+runnumber+R+F+'.fasta'
    
    with open(fastaoutfilename, 'wb') as outfile:
        for filename in glob.glob('temp_fasta/*/*/*.fasta'):
            if filename == fastaoutfilename:
                # don't want to copy the output into the output
                continue
            with open(filename, 'rb') as readfile:
                shutil.copyfileobj(readfile, outfile)
    shutil.rmtree('temp_fasta', ignore_errors=False, onerror=None)
    

    if R!='all':
        rallfs = separated[separated.Reverse_trim == R]
        separated_dic = pd.Series(rallfs.Taxon.values,rallfs.feature_id.values).to_dict()
    else:
        separated_dic = pd.Series(separated.Taxon.values, separated.feature_id.values).to_dict()
    if F!='all':
        fallrs = separated[separated.Forward_trim == F]
        separated_dic = pd.Series(fallrs.Taxon.values,fallrs.feature_id.values).to_dict()
    else:
        separated_dic = pd.Series(separated.Taxon.values, separated.feature_id.values).to_dict()

    fa = SeqIO.parse('input_data/'+community+'/'+fastaoutfilename,
                 "fasta")
    seqs_i_want = [] #we'll put the good sequences here
    for record in fa: #a SeqRecord has the accession as record.id, usually.
        if record.id in separated_dic.keys(): #This is how you check if the accession is in the values of the dict
            seqs_i_want.append(record)
    #Now we can write the list of records to a fasta file. This will take care of the formatting etc
    with open('input_data/'+community+composition+runnumber+R+F+'.fasta', "w") as f:
        SeqIO.write(seqs_i_want, f, "fasta")
    return print('Saved selected sequences as' 'input_data/'+community+'/'+runnumber+composition+'R'+R+'F'+F+'.fasta')
    return (files, separated, separated_dic, merged)

In [34]:
files, separated, separated_dic, merged = divide_by_comm('16S', 'Staggered', '46')

Set up manifest ...
Set up metadata ...
Saved filtering_asvs.tsv


/var/folders/kv/yq6jygk116l27q2bwnk25k0w0000gn/T/ipykernel_95831/3769239956.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  tax["table_id"] = tax["table_id"].str.replace(".tsv", "")
/var/folders/kv/yq6jygk116l27q2bwnk25k0w0000gn/T/ipykernel_95831/3769239956.py:32: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  tax['Forward_trim'], tax['Reverse_trim'] = tax['table_id'].str.split('R', 1).str


Appended all taxonomies to taxos


FileNotFoundError: [Errno 2] No such file or directory: 'input_data/16S/merged.fasta'